In [2]:
import numpy as np
import glob
import os
from PIL import Image
from pprint import pprint
from collections import defaultdict
import pandas as pd
import albumentations as A
import torch
import cv2

In [3]:
class Evaluator(object):
    """
    https://github.com/jfzhang95/pytorch-deeplab-xception/blob/master/utils/metrics.py
    """
    def __init__(self, num_class):
        self.num_class = num_class
        self.confusion_matrix = np.zeros((self.num_class,)*2)

    def Pixel_Accuracy(self):
        Acc = np.diag(self.confusion_matrix).sum() / self.confusion_matrix.sum()
        return Acc

    def Pixel_Accuracy_Class(self):
        Acc = np.diag(self.confusion_matrix) / self.confusion_matrix.sum(axis=1)
        Acc = np.nanmean(Acc)
        return Acc

    def Mean_Intersection_over_Union(self):
        MIoU = np.diag(self.confusion_matrix) / (
                    np.sum(self.confusion_matrix, axis=1) + np.sum(self.confusion_matrix, axis=0) -
                    np.diag(self.confusion_matrix))
        MIoU = np.nanmean(MIoU)
        return MIoU

    def Frequency_Weighted_Intersection_over_Union(self):
        freq = np.sum(self.confusion_matrix, axis=1) / np.sum(self.confusion_matrix)
        iu = np.diag(self.confusion_matrix) / (
                    np.sum(self.confusion_matrix, axis=1) + np.sum(self.confusion_matrix, axis=0) -
                    np.diag(self.confusion_matrix))

        FWIoU = (freq[freq > 0] * iu[freq > 0]).sum()
        return FWIoU, iu

    def _generate_matrix(self, gt_image, pre_image):
        mask = (gt_image >= 0) & (gt_image < self.num_class)
        label = self.num_class * gt_image[mask].astype('int') + pre_image[mask]
        count = np.bincount(label, minlength=self.num_class**2)
        confusion_matrix = count.reshape(self.num_class, self.num_class)
        return confusion_matrix

    def add_batch(self, gt_image, pre_image):
        assert gt_image.shape == pre_image.shape
        self.confusion_matrix += self._generate_matrix(gt_image, pre_image)

    def reset(self):
        self.confusion_matrix = np.zeros((self.num_class,) * 2)

def get_msk(msk_pth):
    msk = np.asarray(Image.open(msk_pth)).copy()
    msk[msk>0] = 1
    return msk

csv_pths = {
    'val0': r'../data/val_0.csv',
    'val1': r'../data/val_1.csv',
    'val2': r'../data/val_2.csv'
}


In [9]:
val = 'val0'
gt_dir = r'/boot/data1/kang_data/zkxt21_seaice/trainData/gt'
# csv_gt_nms = pd.read_csv(val)['label'].tolist()
# gt_pths = list(map(lambda x: os.path.join(gt_dir, x), csv_gt_nms))

model_dict = {
    'EffUNet_stage4_v0': '../submit/EffUNet_stage4/20240109_194957',
    'EffUNet_stage4_v1': '../submit/EffUNet_stage4/20240115_220855',
    'EffUNet_stage4_v2': '../submit/EffUNet_stage4/20240109_195120',
    'EffUNet_stage5_v0': '../submit/EffUNet_stage5/20240118_182135',
    'EffUNet_stage5_v1': '../submit/EffUNet_stage5/20240118_182149',
    'EffUNet_stage5_v2': '../submit/EffUNet_stage5/20240118_182155',
    # 'EffUNet_stage4+ce+Dice+BD_v0': '../submit/EffUNet_stage4+ce+Dice+BD/20240118_202638',
    # 'EffUNet_stage4+ce+Dice+BD_v1': '../submit/EffUNet_stage4+ce+Dice+BD/20240118_202648',
    # 'EffUNet_stage4+ce+Dice+BD_v2': '../submit/EffUNet_stage4+ce+Dice+BD/20240118_202656',
    # 'EffUNet_stage4_deepsupversion_v0': '../submit/EffUNet_stage4_deepsupversion/20240112_154225',
    # 'EffUNet_stage4_deepsupversion_v1': '../submit/EffUNet_stage4_deepsupversion/20240112_154241',
    # 'EffUNet_stage4_deepsupversion_v2': '../submit/EffUNet_stage4_deepsupversion/20240112_154252',
    # 'EffUNet_stage4_WS_v0': '../submit/EffUNet_stage4_WS/20240116_153211',
    # 'EffUNet_stage4_WS_v1': '../submit/EffUNet_stage4_WS/20240116_153219',
    # 'EffUNet_stage4_WS_v2': '../submit/EffUNet_stage4_WS/20240116_153224',
    # 'EffUNet_stage4+ce_v0': '../submit/EffUNet_stage4+ce/20240115_094933',
    # 'EffUNet_stage4+ce_v1': '../submit/EffUNet_stage4+ce/20240116_153005',
    # 'EffUNet_stage4+ce_v2': '../submit/EffUNet_stage4+ce/20240116_153022',
    # 'EffUNet_stage4+ce+boundary_v0': '../submit/EffUNet_stage4+ce+boundary/20240112_101312',
    # 'EffUNet_stage4+ce+boundary_v1': '../submit/EffUNet_stage4+ce+boundary/20240112_101423',
    # 'EffUNet_stage4+ce+boundary_v2': '../submit/EffUNet_stage4+ce+boundary/20240112_101504',
    # 'EffUNet_stage4+ce+cls-dice_v0': '../submit/EffUNet_stage4+ce+cls-dice/20240117_095806',
    # 'EffUNet_stage4+ce+cls-dice_v1': '../submit/EffUNet_stage4+ce+cls-dice/20240116_183245',
    # 'EffUNet_stage4+ce+cls-dice_v2': '../submit/EffUNet_stage4+ce+cls-dice/20240116_183251',
    # 'UNet_v0': '../submit/UNet/20240110_124331',
    # 'UNet_v1': '../submit/UNet/20240110_124414',
    # 'UNet_v2': '../submit/UNet/20240110_124427',
    # 'pspnet_v0': '../submit/pspnet/20240110_192717',
    # 'pspnet_v1': '../submit/pspnet/20240110_192751',
    # 'pspnet_v2': '../submit/pspnet/20240110_192804',
    # 'Linknet_v0': '../submit/Linknet/20240110_124514',
    # 'Linknet_v1': '../submit/Linknet/20240110_124522',
    # 'Linknet_v2': '../submit/Linknet/20240110_124547',
    # 'fpn_v0': '../submit/fpn/20240110_192850',
    # 'fpn_v1': '../submit/fpn/20240110_192913',
    # 'fpn_v2': '../submit/fpn/20240110_192936',
    # 'deeplabv3_v0': '../submit/deeplabv3/20240110_125054',
    # 'deeplabv3_v1': '../submit/deeplabv3/20240110_125135',
    # 'deeplabv3_v2': '../submit/deeplabv3/20240110_125146',
    # 'deeplabv3+_v0': '../submit/deeplabv3+/20240110_125433',
    # 'deeplabv3+_v1': '../submit/deeplabv3+/20240110_125505',
    # 'deeplabv3+_v2': '../submit/deeplabv3+/20240110_125517',
}


In [10]:
results = defaultdict()
evaluator = Evaluator(2)

for method, pred_dir in model_dict.items():
    evaluator.reset()
    print(method)
    results[method] = defaultdict()
    pred_pths = glob.glob(os.path.join(pred_dir, 'test_predict', '*.png'))
    pred_nms = map(os.path.basename, pred_pths)
    gt_pths = [os.path.join(gt_dir, pred_nm) for pred_nm in pred_nms]
    for _ in map(lambda x: evaluator.add_batch(get_msk(x[0]), get_msk(x[1])), zip(gt_pths, pred_pths)):
        pass

    results[method]['FwIoU'], results[method]['IoUs'] = evaluator.Frequency_Weighted_Intersection_over_Union()
    results[method]['OA'] = evaluator.Pixel_Accuracy()
    
    results[method]['FwIoU'] = results[method]['FwIoU']*100
    results[method]['IoUs'] = results[method]['IoUs'][0]*100
    results[method]['OA'] = results[method]['OA']*100
pprint(results)

EffUNet_stage4_v0
EffUNet_stage4_v1
EffUNet_stage4_v2
EffUNet_stage5_v0
EffUNet_stage5_v1
EffUNet_stage5_v2
defaultdict(None,
            {'EffUNet_stage4_v0': defaultdict(None,
                                              {'FwIoU': 98.51314966937313,
                                               'IoUs': 99.15114209133907,
                                               'OA': 99.24319412907944}),
             'EffUNet_stage4_v1': defaultdict(None,
                                              {'FwIoU': 98.50468724082937,
                                               'IoUs': 99.17164420405243,
                                               'OA': 99.23704719543457}),
             'EffUNet_stage4_v2': defaultdict(None,
                                              {'FwIoU': 98.86676078651458,
                                               'IoUs': 99.34860363596844,
                                               'OA': 99.4249496459961}),
             'EffUNet_stage5_v0': defaultdict(None

UNet

In [10]:
results_mean = defaultdict()
results_mean['UNet'] = defaultdict()
results_mean['UNet_std'] = defaultdict()
results_mean['UNet']['FwIoU'] = np.mean([results['UNet_v0']['FwIoU'], results['UNet_v1']['FwIoU'], results['UNet_v2']['FwIoU']])
results_mean['UNet']['IoUs'] = np.mean([results['UNet_v0']['IoUs'], results['UNet_v1']['IoUs'], results['UNet_v2']['IoUs']])
results_mean['UNet']['OA'] = np.mean([results['UNet_v0']['OA'], results['UNet_v1']['OA'], results['UNet_v2']['OA']])

results_mean['UNet_std']['FwIoU'] = np.std([results['UNet_v0']['FwIoU'], results['UNet_v1']['FwIoU'], results['UNet_v2']['FwIoU']])
results_mean['UNet_std']['IoUs'] = np.std([results['UNet_v0']['IoUs'], results['UNet_v1']['IoUs'], results['UNet_v2']['IoUs']])
results_mean['UNet_std']['OA'] = np.std([results['UNet_v0']['OA'], results['UNet_v1']['OA'], results['UNet_v2']['OA']])

pprint(results_mean)

defaultdict(None,
            {'UNet': defaultdict(None,
                                 {'FwIoU': 98.30736227129074,
                                  'IoUs': 99.04099769131265,
                                  'OA': 99.1359151204427}),
             'UNet_std': defaultdict(None,
                                     {'FwIoU': 0.23948698368356863,
                                      'IoUs': 0.12020955942171486,
                                      'OA': 0.12393343850150013})})


PSPnet

In [11]:
results_mean = defaultdict()
results_mean['pspnet'] = defaultdict()
results_mean['pspnet_std'] = defaultdict()
results_mean['pspnet']['FwIoU'] = np.mean([results['pspnet_v0']['FwIoU'], results['pspnet_v1']['FwIoU'], results['pspnet_v2']['FwIoU']])
results_mean['pspnet']['IoUs'] = np.mean([results['pspnet_v0']['IoUs'], results['pspnet_v1']['IoUs'], results['pspnet_v2']['IoUs']])
results_mean['pspnet']['OA'] = np.mean([results['pspnet_v0']['OA'], results['pspnet_v1']['OA'], results['pspnet_v2']['OA']])

results_mean['pspnet_std']['FwIoU'] = np.std([results['pspnet_v0']['FwIoU'], results['pspnet_v1']['FwIoU'], results['pspnet_v2']['FwIoU']])
results_mean['pspnet_std']['IoUs'] = np.std([results['pspnet_v0']['IoUs'], results['pspnet_v1']['IoUs'], results['pspnet_v2']['IoUs']])
results_mean['pspnet_std']['OA'] = np.std([results['pspnet_v0']['OA'], results['pspnet_v1']['OA'], results['pspnet_v2']['OA']])

pprint(results_mean)

defaultdict(None,
            {'pspnet': defaultdict(None,
                                   {'FwIoU': 97.64017555911126,
                                    'IoUs': 98.63911705504573,
                                    'OA': 98.77610079447429}),
             'pspnet_std': defaultdict(None,
                                       {'FwIoU': 0.22167427345983545,
                                        'IoUs': 0.1264901547102529,
                                        'OA': 0.11812241817014042})})


Linknet

In [13]:
results_mean = defaultdict()
results_mean['Linknet'] = defaultdict()
results_mean['Linknet_std'] = defaultdict()
results_mean['Linknet']['FwIoU'] = np.mean([results['Linknet_v0']['FwIoU'], results['Linknet_v1']['FwIoU'], results['Linknet_v2']['FwIoU']])
results_mean['Linknet']['IoUs'] = np.mean([results['Linknet_v0']['IoUs'], results['Linknet_v1']['IoUs'], results['Linknet_v2']['IoUs']])
results_mean['Linknet']['OA'] = np.mean([results['Linknet_v0']['OA'], results['Linknet_v1']['OA'], results['Linknet_v2']['OA']])

results_mean['Linknet_std']['FwIoU'] = np.std([results['Linknet_v0']['FwIoU'], results['Linknet_v1']['FwIoU'], results['Linknet_v2']['FwIoU']])
results_mean['Linknet_std']['IoUs'] = np.std([results['Linknet_v0']['IoUs'], results['Linknet_v1']['IoUs'], results['Linknet_v2']['IoUs']])
results_mean['Linknet_std']['OA'] = np.std([results['Linknet_v0']['OA'], results['Linknet_v1']['OA'], results['Linknet_v2']['OA']])

pprint(results_mean)

defaultdict(None,
            {'Linknet': defaultdict(None,
                                    {'FwIoU': 97.92983560872214,
                                     'IoUs': 98.82823497552249,
                                     'OA': 98.94251950581868}),
             'Linknet_std': defaultdict(None,
                                        {'FwIoU': 0.23152258969594763,
                                         'IoUs': 0.10845293047641726,
                                         'OA': 0.1181137637451888})})


FPN

In [14]:
results_mean = defaultdict()
results_mean['fpn'] = defaultdict()
results_mean['fpn_std'] = defaultdict()
results_mean['fpn']['FwIoU'] = np.mean([results['fpn_v0']['FwIoU'], results['fpn_v1']['FwIoU'], results['fpn_v2']['FwIoU']])
results_mean['fpn']['IoUs'] = np.mean([results['fpn_v0']['IoUs'], results['fpn_v1']['IoUs'], results['fpn_v2']['IoUs']])
results_mean['fpn']['OA'] = np.mean([results['fpn_v0']['OA'], results['fpn_v1']['OA'], results['fpn_v2']['OA']])

results_mean['fpn_std']['FwIoU'] = np.std([results['fpn_v0']['FwIoU'], results['fpn_v1']['FwIoU'], results['fpn_v2']['FwIoU']])
results_mean['fpn_std']['IoUs'] = np.std([results['fpn_v0']['IoUs'], results['fpn_v1']['IoUs'], results['fpn_v2']['IoUs']])
results_mean['fpn_std']['OA'] = np.std([results['fpn_v0']['OA'], results['fpn_v1']['OA'], results['fpn_v2']['OA']])

pprint(results_mean)

defaultdict(None,
            {'fpn': defaultdict(None,
                                {'FwIoU': 97.90400455819388,
                                 'IoUs': 98.80249155897404,
                                 'OA': 98.92112414042155}),
             'fpn_std': defaultdict(None,
                                    {'FwIoU': 0.3191313502374523,
                                     'IoUs': 0.1744660754827764,
                                     'OA': 0.1681175746616953})})


Deeplabv3

In [15]:
results_mean = defaultdict()
results_mean['deeplabv3'] = defaultdict()
results_mean['deeplabv3_std'] = defaultdict()
results_mean['deeplabv3']['FwIoU'] = np.mean([results['deeplabv3_v0']['FwIoU'], results['deeplabv3_v1']['FwIoU'], results['deeplabv3_v2']['FwIoU']])
results_mean['deeplabv3']['IoUs'] = np.mean([results['deeplabv3_v0']['IoUs'], results['deeplabv3_v1']['IoUs'], results['deeplabv3_v2']['IoUs']])
results_mean['deeplabv3']['OA'] = np.mean([results['deeplabv3_v0']['OA'], results['deeplabv3_v1']['OA'], results['deeplabv3_v2']['OA']])

results_mean['deeplabv3_std']['FwIoU'] = np.std([results['deeplabv3_v0']['FwIoU'], results['deeplabv3_v1']['FwIoU'], results['deeplabv3_v2']['FwIoU']])
results_mean['deeplabv3_std']['IoUs'] = np.std([results['deeplabv3_v0']['IoUs'], results['deeplabv3_v1']['IoUs'], results['deeplabv3_v2']['IoUs']])
results_mean['deeplabv3_std']['OA'] = np.std([results['deeplabv3_v0']['OA'], results['deeplabv3_v1']['OA'], results['deeplabv3_v2']['OA']])

pprint(results_mean)

defaultdict(None,
            {'deeplabv3': defaultdict(None,
                                      {'FwIoU': 97.64147805590007,
                                       'IoUs': 98.64448564167897,
                                       'OA': 98.77965990702312}),
             'deeplabv3_std': defaultdict(None,
                                          {'FwIoU': 0.14310394423943726,
                                           'IoUs': 0.06089735003408994,
                                           'OA': 0.07657296787873133})})


Deeplabv3+

In [16]:
results_mean = defaultdict()
results_mean['deeplabv3+'] = defaultdict()
results_mean['deeplabv3+_std'] = defaultdict()
results_mean['deeplabv3+']['FwIoU'] = np.mean([results['deeplabv3+_v0']['FwIoU'], results['deeplabv3+_v1']['FwIoU'], results['deeplabv3+_v2']['FwIoU']])
results_mean['deeplabv3+']['IoUs'] = np.mean([results['deeplabv3+_v0']['IoUs'], results['deeplabv3+_v1']['IoUs'], results['deeplabv3+_v2']['IoUs']])
results_mean['deeplabv3+']['OA'] = np.mean([results['deeplabv3+_v0']['OA'], results['deeplabv3+_v1']['OA'], results['deeplabv3+_v2']['OA']])

results_mean['deeplabv3+_std']['FwIoU'] = np.std([results['deeplabv3+_v0']['FwIoU'], results['deeplabv3+_v1']['FwIoU'], results['deeplabv3+_v2']['FwIoU']])
results_mean['deeplabv3+_std']['IoUs'] = np.std([results['deeplabv3+_v0']['IoUs'], results['deeplabv3+_v1']['IoUs'], results['deeplabv3+_v2']['IoUs']])
results_mean['deeplabv3+_std']['OA'] = np.std([results['deeplabv3+_v0']['OA'], results['deeplabv3+_v1']['OA'], results['deeplabv3+_v2']['OA']])

pprint(results_mean)

defaultdict(None,
            {'deeplabv3+': defaultdict(None,
                                       {'FwIoU': 97.80694300545083,
                                        'IoUs': 98.74568310677496,
                                        'OA': 98.87060801188152}),
             'deeplabv3+_std': defaultdict(None,
                                           {'FwIoU': 0.14130105905871165,
                                            'IoUs': 0.07075735483999772,
                                            'OA': 0.0761164693720606})})


EffUNet_stage4

In [8]:
results_mean = defaultdict()
results_mean['EffUNet_stage4'] = defaultdict()
results_mean['EffUNet_stage4_std'] = defaultdict()
results_mean['EffUNet_stage4']['FwIoU'] = np.mean([results['EffUNet_stage4_v0']['FwIoU'], results['EffUNet_stage4_v1']['FwIoU'], results['EffUNet_stage4_v2']['FwIoU']])
results_mean['EffUNet_stage4']['IoUs'] = np.mean([results['EffUNet_stage4_v0']['IoUs'], results['EffUNet_stage4_v1']['IoUs'], results['EffUNet_stage4_v2']['IoUs']])
results_mean['EffUNet_stage4']['OA'] = np.mean([results['EffUNet_stage4_v0']['OA'], results['EffUNet_stage4_v1']['OA'], results['EffUNet_stage4_v2']['OA']])

results_mean['EffUNet_stage4_std']['FwIoU'] = np.std([results['EffUNet_stage4_v0']['FwIoU'], results['EffUNet_stage4_v1']['FwIoU'], results['EffUNet_stage4_v2']['FwIoU']])
results_mean['EffUNet_stage4_std']['IoUs'] = np.std([results['EffUNet_stage4_v0']['IoUs'], results['EffUNet_stage4_v1']['IoUs'], results['EffUNet_stage4_v2']['IoUs']])
results_mean['EffUNet_stage4_std']['OA'] = np.std([results['EffUNet_stage4_v0']['OA'], results['EffUNet_stage4_v1']['OA'], results['EffUNet_stage4_v2']['OA']])

pprint(results_mean)

defaultdict(None,
            {'EffUNet_stage4': defaultdict(None,
                                           {'FwIoU': 98.62819923223903,
                                            'IoUs': 99.22379664378666,
                                            'OA': 99.30173032350336}),
             'EffUNet_stage4_std': defaultdict(None,
                                               {'FwIoU': 0.16872386623906888,
                                                'IoUs': 0.08864789198293961,
                                                'OA': 0.08716534967131259})})


EffUNet_stage5

In [11]:
results_mean = defaultdict()
results_mean['EffUNet_stage5'] = defaultdict()
results_mean['EffUNet_stage5_std'] = defaultdict()
results_mean['EffUNet_stage5']['FwIoU'] = np.mean([results['EffUNet_stage5_v0']['FwIoU'], results['EffUNet_stage5_v1']['FwIoU'], results['EffUNet_stage5_v2']['FwIoU']])
results_mean['EffUNet_stage5']['IoUs'] = np.mean([results['EffUNet_stage5_v0']['IoUs'], results['EffUNet_stage5_v1']['IoUs'], results['EffUNet_stage5_v2']['IoUs']])
results_mean['EffUNet_stage5']['OA'] = np.mean([results['EffUNet_stage5_v0']['OA'], results['EffUNet_stage5_v1']['OA'], results['EffUNet_stage5_v2']['OA']])

results_mean['EffUNet_stage5_std']['FwIoU'] = np.std([results['EffUNet_stage5_v0']['FwIoU'], results['EffUNet_stage5_v1']['FwIoU'], results['EffUNet_stage5_v2']['FwIoU']])
results_mean['EffUNet_stage5_std']['IoUs'] = np.std([results['EffUNet_stage5_v0']['IoUs'], results['EffUNet_stage5_v1']['IoUs'], results['EffUNet_stage5_v2']['IoUs']])
results_mean['EffUNet_stage5_std']['OA'] = np.std([results['EffUNet_stage5_v0']['OA'], results['EffUNet_stage5_v1']['OA'], results['EffUNet_stage5_v2']['OA']])

pprint(results_mean)

defaultdict(None,
            {'EffUNet_stage5': defaultdict(None,
                                           {'FwIoU': 98.53982205496355,
                                            'IoUs': 99.17290305257683,
                                            'OA': 99.25597190856934}),
             'EffUNet_stage5_std': defaultdict(None,
                                               {'FwIoU': 0.17937365270492397,
                                                'IoUs': 0.09190625845514834,
                                                'OA': 0.09304454298473747})})


EffUNet_stage4+ce+Dice+BD

In [8]:
results_mean = defaultdict()
results_mean['EffUNet_stage4+ce+Dice+BD'] = defaultdict()
results_mean['EffUNet_stage4+ce+Dice+BD_std'] = defaultdict()
results_mean['EffUNet_stage4+ce+Dice+BD']['FwIoU'] = np.mean([results['EffUNet_stage4+ce+Dice+BD_v0']['FwIoU'], results['EffUNet_stage4+ce+Dice+BD_v1']['FwIoU'], results['EffUNet_stage4+ce+Dice+BD_v2']['FwIoU']])
results_mean['EffUNet_stage4+ce+Dice+BD']['IoUs'] = np.mean([results['EffUNet_stage4+ce+Dice+BD_v0']['IoUs'], results['EffUNet_stage4+ce+Dice+BD_v1']['IoUs'], results['EffUNet_stage4+ce+Dice+BD_v2']['IoUs']])
results_mean['EffUNet_stage4+ce+Dice+BD']['OA'] = np.mean([results['EffUNet_stage4+ce+Dice+BD_v0']['OA'], results['EffUNet_stage4+ce+Dice+BD_v1']['OA'], results['EffUNet_stage4+ce+Dice+BD_v2']['OA']])

results_mean['EffUNet_stage4+ce+Dice+BD_std']['FwIoU'] = np.std([results['EffUNet_stage4+ce+Dice+BD_v0']['FwIoU'], results['EffUNet_stage4+ce+Dice+BD_v1']['FwIoU'], results['EffUNet_stage4+ce+Dice+BD_v2']['FwIoU']])
results_mean['EffUNet_stage4+ce+Dice+BD_std']['IoUs'] = np.std([results['EffUNet_stage4+ce+Dice+BD_v0']['IoUs'], results['EffUNet_stage4+ce+Dice+BD_v1']['IoUs'], results['EffUNet_stage4+ce+Dice+BD_v2']['IoUs']])
results_mean['EffUNet_stage4+ce+Dice+BD_std']['OA'] = np.std([results['EffUNet_stage4+ce+Dice+BD_v0']['OA'], results['EffUNet_stage4+ce+Dice+BD_v1']['OA'], results['EffUNet_stage4+ce+Dice+BD_v2']['OA']])

pprint(results_mean)

defaultdict(None,
            {'EffUNet_stage4+ce+Dice+BD': defaultdict(None,
                                                      {'FwIoU': 98.50617697021694,
                                                       'IoUs': 99.15417296450379,
                                                       'OA': 99.23891003926595}),
             'EffUNet_stage4+ce+Dice+BD_std': defaultdict(None,
                                                          {'FwIoU': 0.20603376369164889,
                                                           'IoUs': 0.10821956343323594,
                                                           'OA': 0.1064926552001793})})


EffUNet_stage4_WS

In [13]:
results_mean = defaultdict()
results_mean['EffUNet_stage4_WS'] = defaultdict()
results_mean['EffUNet_stage4_WS_std'] = defaultdict()
results_mean['EffUNet_stage4_WS']['FwIoU'] = np.mean([results['EffUNet_stage4_WS_v0']['FwIoU'], results['EffUNet_stage4_WS_v1']['FwIoU'], results['EffUNet_stage4_WS_v2']['FwIoU']])
results_mean['EffUNet_stage4_WS']['IoUs'] = np.mean([results['EffUNet_stage4_WS_v0']['IoUs'], results['EffUNet_stage4_WS_v1']['IoUs'], results['EffUNet_stage4_WS_v2']['IoUs']])
results_mean['EffUNet_stage4_WS']['OA'] = np.mean([results['EffUNet_stage4_WS_v0']['OA'], results['EffUNet_stage4_WS_v1']['OA'], results['EffUNet_stage4_WS_v2']['OA']])

results_mean['EffUNet_stage4_WS_std']['FwIoU'] = np.std([results['EffUNet_stage4_WS_v0']['FwIoU'], results['EffUNet_stage4_WS_v1']['FwIoU'], results['EffUNet_stage4_WS_v2']['FwIoU']])
results_mean['EffUNet_stage4_WS_std']['IoUs'] = np.std([results['EffUNet_stage4_WS_v0']['IoUs'], results['EffUNet_stage4_WS_v1']['IoUs'], results['EffUNet_stage4_WS_v2']['IoUs']])
results_mean['EffUNet_stage4_WS_std']['OA'] = np.std([results['EffUNet_stage4_WS_v0']['OA'], results['EffUNet_stage4_WS_v1']['OA'], results['EffUNet_stage4_WS_v2']['OA']])

pprint(results_mean)

defaultdict(None,
            {'EffUNet_stage4_WS': defaultdict(None,
                                              {'FwIoU': 98.52659453906642,
                                               'IoUs': 99.16104924529436,
                                               'OA': 99.24648666381836}),
             'EffUNet_stage4_WS_std': defaultdict(None,
                                                  {'FwIoU': 0.18588921519949667,
                                                   'IoUs': 0.10238281703656259,
                                                   'OA': 0.09678915549542857})})


EffUNet_stage4_deepsupversion

In [37]:
results_mean = defaultdict()
results_mean['EffUNet_stage4_deepsupversion'] = defaultdict()
results_mean['EffUNet_stage4_deepsupversion_std'] = defaultdict()
results_mean['EffUNet_stage4_deepsupversion']['FwIoU'] = np.mean([results['EffUNet_stage4_deepsupversion_v0']['FwIoU'], results['EffUNet_stage4_deepsupversion_v1']['FwIoU'], results['EffUNet_stage4_deepsupversion_v2']['FwIoU']])
results_mean['EffUNet_stage4_deepsupversion']['IoUs'] = np.mean([results['EffUNet_stage4_deepsupversion_v0']['IoUs'], results['EffUNet_stage4_deepsupversion_v1']['IoUs'], results['EffUNet_stage4_deepsupversion_v2']['IoUs']])
results_mean['EffUNet_stage4_deepsupversion']['OA'] = np.mean([results['EffUNet_stage4_deepsupversion_v0']['OA'], results['EffUNet_stage4_deepsupversion_v1']['OA'], results['EffUNet_stage4_deepsupversion_v2']['OA']])

results_mean['EffUNet_stage4_deepsupversion_std']['FwIoU'] = np.std([results['EffUNet_stage4_deepsupversion_v0']['FwIoU'], results['EffUNet_stage4_deepsupversion_v1']['FwIoU'], results['EffUNet_stage4_deepsupversion_v2']['FwIoU']])
results_mean['EffUNet_stage4_deepsupversion_std']['IoUs'] = np.std([results['EffUNet_stage4_deepsupversion_v0']['IoUs'], results['EffUNet_stage4_deepsupversion_v1']['IoUs'], results['EffUNet_stage4_deepsupversion_v2']['IoUs']])
results_mean['EffUNet_stage4_deepsupversion_std']['OA'] = np.std([results['EffUNet_stage4_deepsupversion_v0']['OA'], results['EffUNet_stage4_deepsupversion_v1']['OA'], results['EffUNet_stage4_deepsupversion_v2']['OA']])

pprint(results_mean)

defaultdict(None,
            {'EffUNet_stage4_deepsupversion': defaultdict(None,
                                                          {'FwIoU': 98.58055786917144,
                                                           'IoUs': 99.1955375621428,
                                                           'OA': 99.27668317159016}),
             'EffUNet_stage4_deepsupversion_std': defaultdict(None,
                                                              {'FwIoU': 0.1914958295212156,
                                                               'IoUs': 0.1024807097787672,
                                                               'OA': 0.09910256507092348})})


EffUNet_stage4+ce

In [8]:
results_mean = defaultdict()
results_mean['EffUNet_stage4+ce'] = defaultdict()
results_mean['EffUNet_stage4+ce_std'] = defaultdict()
results_mean['EffUNet_stage4+ce']['FwIoU'] = np.mean([results['EffUNet_stage4+ce_v0']['FwIoU'], results['EffUNet_stage4+ce_v1']['FwIoU'], results['EffUNet_stage4+ce_v2']['FwIoU']])
results_mean['EffUNet_stage4+ce']['IoUs'] = np.mean([results['EffUNet_stage4+ce_v0']['IoUs'], results['EffUNet_stage4+ce_v1']['IoUs'], results['EffUNet_stage4+ce_v2']['IoUs']])
results_mean['EffUNet_stage4+ce']['OA'] = np.mean([results['EffUNet_stage4+ce_v0']['OA'], results['EffUNet_stage4+ce_v1']['OA'], results['EffUNet_stage4+ce_v2']['OA']])

results_mean['EffUNet_stage4+ce_std']['FwIoU'] = np.std([results['EffUNet_stage4+ce_v0']['FwIoU'], results['EffUNet_stage4+ce_v1']['FwIoU'], results['EffUNet_stage4+ce_v2']['FwIoU']])
results_mean['EffUNet_stage4+ce_std']['IoUs'] = np.std([results['EffUNet_stage4+ce_v0']['IoUs'], results['EffUNet_stage4+ce_v1']['IoUs'], results['EffUNet_stage4+ce_v2']['IoUs']])
results_mean['EffUNet_stage4+ce_std']['OA'] = np.std([results['EffUNet_stage4+ce_v0']['OA'], results['EffUNet_stage4+ce_v1']['OA'], results['EffUNet_stage4+ce_v2']['OA']])

pprint(results_mean)

defaultdict(None,
            {'EffUNet_stage4+ce': defaultdict(None,
                                              {'FwIoU': 98.55124335598519,
                                               'IoUs': 99.17908402803657,
                                               'OA': 99.26179313659668}),
             'EffUNet_stage4+ce_std': defaultdict(None,
                                                  {'FwIoU': 0.19467928378020713,
                                                   'IoUs': 0.10367117980742635,
                                                   'OA': 0.10042573623861893})})


EffUNet_stage4+ce+boundary

In [25]:
results_mean = defaultdict()
results_mean['EffUNet_stage4+ce+boundary'] = defaultdict()
results_mean['EffUNet_stage4+ce+boundary_std'] = defaultdict()
results_mean['EffUNet_stage4+ce+boundary']['FwIoU'] = np.mean([results['EffUNet_stage4+ce+boundary_v0']['FwIoU'], results['EffUNet_stage4+ce+boundary_v1']['FwIoU'], results['EffUNet_stage4+ce+boundary_v2']['FwIoU']])
results_mean['EffUNet_stage4+ce+boundary']['IoUs'] = np.mean([results['EffUNet_stage4+ce+boundary_v0']['IoUs'], results['EffUNet_stage4+ce+boundary_v1']['IoUs'], results['EffUNet_stage4+ce+boundary_v2']['IoUs']])
results_mean['EffUNet_stage4+ce+boundary']['OA'] = np.mean([results['EffUNet_stage4+ce+boundary_v0']['OA'], results['EffUNet_stage4+ce+boundary_v1']['OA'], results['EffUNet_stage4+ce+boundary_v2']['OA']])

results_mean['EffUNet_stage4+ce+boundary_std']['FwIoU'] = np.std([results['EffUNet_stage4+ce+boundary_v0']['FwIoU'], results['EffUNet_stage4+ce+boundary_v1']['FwIoU'], results['EffUNet_stage4+ce+boundary_v2']['FwIoU']])
results_mean['EffUNet_stage4+ce+boundary_std']['IoUs'] = np.std([results['EffUNet_stage4+ce+boundary_v0']['IoUs'], results['EffUNet_stage4+ce+boundary_v1']['IoUs'], results['EffUNet_stage4+ce+boundary_v2']['IoUs']])
results_mean['EffUNet_stage4+ce+boundary_std']['OA'] = np.std([results['EffUNet_stage4+ce+boundary_v0']['OA'], results['EffUNet_stage4+ce+boundary_v1']['OA'], results['EffUNet_stage4+ce+boundary_v2']['OA']])

pprint(results_mean)

defaultdict(None,
            {'EffUNet_stage4+ce+boundary': defaultdict(None,
                                                       {'FwIoU': 98.58781629956472,
                                                        'IoUs': 99.20020389083476,
                                                        'OA': 99.28083992004395}),
             'EffUNet_stage4+ce+boundary_std': defaultdict(None,
                                                           {'FwIoU': 0.18164812314853634,
                                                            'IoUs': 0.09662826974973131,
                                                            'OA': 0.09405595451922118})})


EffUNet_stage4+ce+cls-dice

In [7]:
results_mean = defaultdict()
results_mean['EffUNet_stage4+ce+cls-dice'] = defaultdict()
results_mean['EffUNet_stage4+ce+cls-dice_std'] = defaultdict()
results_mean['EffUNet_stage4+ce+cls-dice']['FwIoU'] = np.mean([results['EffUNet_stage4+ce+cls-dice_v0']['FwIoU'], results['EffUNet_stage4+ce+cls-dice_v1']['FwIoU'], results['EffUNet_stage4+ce+cls-dice_v2']['FwIoU']])
results_mean['EffUNet_stage4+ce+cls-dice']['IoUs'] = np.mean([results['EffUNet_stage4+ce+cls-dice_v0']['IoUs'], results['EffUNet_stage4+ce+cls-dice_v1']['IoUs'], results['EffUNet_stage4+ce+cls-dice_v2']['IoUs']])
results_mean['EffUNet_stage4+ce+cls-dice']['OA'] = np.mean([results['EffUNet_stage4+ce+cls-dice_v0']['OA'], results['EffUNet_stage4+ce+cls-dice_v1']['OA'], results['EffUNet_stage4+ce+cls-dice_v2']['OA']])

results_mean['EffUNet_stage4+ce+cls-dice_std']['FwIoU'] = np.std([results['EffUNet_stage4+ce+cls-dice_v0']['FwIoU'], results['EffUNet_stage4+ce+cls-dice_v1']['FwIoU'], results['EffUNet_stage4+ce+cls-dice_v2']['FwIoU']])
results_mean['EffUNet_stage4+ce+cls-dice_std']['IoUs'] = np.std([results['EffUNet_stage4+ce+cls-dice_v0']['IoUs'], results['EffUNet_stage4+ce+cls-dice_v1']['IoUs'], results['EffUNet_stage4+ce+cls-dice_v2']['IoUs']])
results_mean['EffUNet_stage4+ce+cls-dice_std']['OA'] = np.std([results['EffUNet_stage4+ce+cls-dice_v0']['OA'], results['EffUNet_stage4+ce+cls-dice_v1']['OA'], results['EffUNet_stage4+ce+cls-dice_v2']['OA']])

pprint(results_mean)

defaultdict(None,
            {'EffUNet_stage4+ce+cls-dice': defaultdict(None,
                                                       {'FwIoU': 98.57515759713898,
                                                        'IoUs': 99.19209015719377,
                                                        'OA': 99.27373631795247}),
             'EffUNet_stage4+ce+cls-dice_std': defaultdict(None,
                                                           {'FwIoU': 0.16230160310225092,
                                                            'IoUs': 0.0852153001022707,
                                                            'OA': 0.08446306117084498})})
